In [ ]:
import joblib  # Para guardar y cargar los modelos
import numpy as np  # Para trabajo de datos numéricos
import pandas as pd  # Para manipulación y análisis de los datos

# Scikit-Learn
from sklearn.ensemble import RandomForestClassifier # Modelo de ML
from sklearn.metrics import classification_report # Para ver resultados del modelo
from sklearn.model_selection import train_test_split # Para división de datos de entrenamiento y prueba
from sklearn.preprocessing import RobustScaler # Para escalar los datos


In [ ]:
# Cargar datos con fallas ficticias
df = pd.read_csv("datos_con_fallas_sinteticas_vf.csv")

# Lista de sensores para definir falla
sensores = [
    'CNN-3200-CR_0001_MO.PWR',
    'CNN-3200-CR_0001_MO.CUR',
    'CNN-3200-FIT32053.PV',
    'CNN-3200-FIT32054.PV',
    'CNN-3200-PIT32031.PV',
    'CNN-3200-PIT32043.PV',
    'CNN-3200-PIT32056.PV',
    'CNN-3200-TIT32045.PV',
    'CNN-3200-TIT32046.PV'
]

In [ ]:
# Target
target = 'Posible_Falla'

for sensor in sensores:
    sensor_std = f"{sensor}_std"
    nombre = sensor.replace('.', '_')  # Para guardar los archivos del modelo de cada sensor

    print(f"\n... Entrenando modelo para: {sensor} ...")

    # Extraer datos del sensor y su std
    X = df[[sensor, sensor_std]].copy()
    y = df[target].copy()

    # Escalar datos
    scaler = RobustScaler()
    X_scaled = scaler.fit_transform(X)

    # Guardar el scaler con el nombre del sensor correspondiente
    joblib.dump(scaler, f"scaler_{nombre}.pkl")

    # División de datos
    X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

    # Entrenamiento del modelo 
    rf = RandomForestClassifier(n_estimators=100, random_state=42)
    rf.fit(X_train, y_train)

    # Guardar el modelo con el nombre del sensor correspondiente
    joblib.dump(rf, f"modelo_{nombre}.pkl")

    # Evaluación del modelo actual
    y_pred = rf.predict(X_test)
    print(classification_report(y_test, y_pred))
